In [3]:
import sys, os
import math
import pycodec2
import wave
import struct
import pika
import qmesh_common
import qmesh_pb2
import numpy as np
import time

In [13]:
frames_per_pkt = 4
frame_len_ms = 40
frame_len_s = frame_len_ms/1000
sample_rate = int(8e3)
num_channels = 1
c2_bitrate = 1200
c2 = pycodec2.Codec2(c2_bitrate)
size_in_bits = {450    : 18,
                700    : 28,
                1200   : 48,
                1300   : 52,
                1600   : 64,
                2400   : 96,
                3200   : 128}
size_in_bytes = { k:int(math.ceil(float(v)/8.0)) for (k,v) in size_in_bits.items() }
WAV_STRUCT_FORMAT = '{}h'.format(c2.samples_per_frame())
C2_STRUCT_FORMAT = '{}h'.format(size_in_bytes[c2_bitrate])


In [30]:
# Sanity check that the virtualized audio still works

import sounddevice as sd
import soundfile as sf

filename = './sample_audio/codec2_1.wav'
# Extract data and sampling rate from file
data, fs = sf.read(filename, dtype='float32')  
sd.play(data, fs)
status = sd.wait()  # Wait until file is done playing


ALSA lib pcm.c:8526:(snd_pcm_recover) underrun occurred


In [29]:
# Open up the WAV file and process it to codec2
samples = []
wav_file_name = './sample_audio/codec2_1.wav'
wav_file = wave.open(wav_file_name, 'rb')
assert(wav_file.getnchannels() == 1)
assert(wav_file.getsampwidth() == 2)
print("Framerate is %s" % (wav_file.getframerate()))
assert(wav_file.getframerate() == sample_rate)
c2_frames = bytes()
while True:
    frames = wav_file.readframes(int(sample_rate*frame_len_s))
    assert(len(frames)/wav_file.getsampwidth() <= 320)
    if len(frames)/wav_file.getsampwidth() != int(sample_rate*frame_len_s):
        print("Done with WAV file!")
        break
    samples = np.array(struct.unpack(WAV_STRUCT_FORMAT, frames), dtype=np.int16)
    c2_frames += c2.encode(samples)
    if len(c2_frames) == frames_per_pkt*size_in_bytes[c2_bitrate]:
        print('--------')
        print(c2_frames)
        c2_frames = bytes()

Framerate is 8000
--------
b'\x16[\x956\xaf0\x08A50/d\xdezu6g\xe2\xfac\x957{0'
--------
b'\x0cm\x155.D\x19\xff\x156/\xc4\x02P\xe55\xf9z\xdc\xf4\xd90\xceP'
--------
b'\xe2\xf9d\x8a\xdf\xf2\xc2\xf7\xb00(\x88\xf5\xf7\xbd\x94\xdf@\xdd\xf9\xb5\xd0\xfc\xb0'
--------
b'\xc4\xfe\xde\x10Y\x1a\xd3\xf4`\x10\xd0~\xfb\xaaO\x00\xae\xb0\xf2\xd7&\xd2\xbb\xf8'
--------
b'\xf0\xf7u)\x98\x02\xf4\xfe\xf5.\xa7F\xc9\xbe\xf94\xa1\x0e\xfcv\x96\x93\xe9('
--------
b'\xa1\xe9\r \xcd\xde\xe2\xf9\x15/A\xd2\xd5~\x93\x12\xd8\xa2\xdf\xf9\x0e\xaa\xa1\x84'
--------
b'\xf3umI\x9e\x96\xddw\x8e\x1d\x8df\xc0\xf7z(\xce^\xce\xf5\xb7\x08W\x8e'
--------
b'\xfe\xf5w\n_\x04\xc1\xf9\xd2#eR\x02N\x18\x87\xf1\x0c\x01\x88\xf8\x83\xdc\xa4'
--------
b'\xf6\xa0\xcb \x89\xea\t\xdb\x9d\xad\x85\xb0\xdd\xf60|]\xbe\xcc\xfa\xb1H\x87\xa0'
--------
b'\xea\xfa\xbf\xcdf4\xea\xfa\xb5\xca_\x8e\xec\xfa\xb6\xad0\xa2\xea\xf8\xf4\xf66l'
--------
b"\xc2\xf0t\xf0'h\xc7\xf2\x19\x11o\x1c\xdfx\xd3\x10\x83\xe0\xc491\xaaS\x14"
--------
b'\xd8\xf3\xb1\xad\x08p

In [7]:
# Save the encoded frames to a file
c2_file_name = './sample_audio/codec2_1.c2'
c2_file = open(c2_file_name, 'w')
for c2_frame in c2_frames:
    print(c2_frame)
    #c2_file.write(struct.pack(c2_frame, 

b'\x10\xba\xe56\xaf0'
b'%\x8e\xd50/d'
b'\xdezu6g\xe2'
b'\xfac\x957{0'
b'\x0cm\x155.D'
b'\x19\xff\x156/\xc4'
b'\x02P\xe55\xf9z'
b'\xdc\xf4\xd90\xceP'
b'\xe2\xf9d\x8a\xdf\xf2'
b'\xc2\xf7\xb00(\x88'
b'\xf5\xf7\xbd\x94\xdf@'
b'\xdd\xf9\xb5\xd0\xfc\xb0'
b'\xc4\xfe\xde\x10Y\x1a'
b'\xd3\xf4`\x10\xd0~'
b'\xfb\xaaO\x00\xae\xb0'
b'\xf2\xd7&\xd2\xbb\xf8'
b'\xf0\xf7u)\x98\x02'
b'\xf4\xfe\xf5.\xa7F'
b'\xc9\xbe\xf94\xa1\x0e'
b'\xfcv\x96\x93\xe9('
b'\xa1\xe9\r \xcd\xde'
b'\xe2\xf9\x15/A\xd2'
b'\xd5~\x93\x12\xd8\xa2'
b'\xdf\xf9\x0e\xaa\xa1\x84'
b'\xf3umI\x9e\x96'
b'\xddw\x8e\x1d\x8df'
b'\xc0\xf7z(\xce^'
b'\xce\xf5\xb7\x08W\x8e'
b'\xfe\xf5w\n_\x04'
b'\xc1\xf9\xd2#eR'
b'\x02N\x18\x87\xf1\x0c'
b'\x01\x88\xf8\x83\xdc\xa4'
b'\xf6\xa0\xcb \x89\xea'
b'\t\xdb\x9d\xad\x85\xb0'
b'\xdd\xf60|]\xbe'
b'\xcc\xfa\xb1H\x87\xa0'
b'\xea\xfa\xbf\xcdf4'
b'\xea\xfa\xb5\xca_\x8e'
b'\xec\xfa\xb6\xad0\xa2'
b'\xea\xf8\xf4\xf66l'
b"\xc2\xf0t\xf0'h"
b'\xc7\xf2\x19\x11o\x1c'
b'\xdfx\xd3\x10\x83\xe0'
b'\xc491\xaaS\x14'
b'\xd8\xf3\

In [8]:
# Open the file, load up the frames

In [9]:
# Do the codec2 decode

In [10]:
# Save the WAV to disk